## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Coahuayana,MX,18.73,-103.68,89.10,50,18,5.59,few clouds
1,1,Lompoc,US,34.64,-120.46,55.40,66,75,19.46,broken clouds
2,2,Dulce Nombre De Culmi,HN,15.10,-85.53,86.00,74,75,1.12,broken clouds
3,3,Puerto Escondido,MX,15.85,-97.07,87.80,66,40,5.82,scattered clouds
4,4,Saint-Philippe,RE,-21.36,55.77,73.40,78,0,8.05,clear sky
5,5,Mataura,NZ,-46.19,168.86,50.00,92,100,2.04,overcast clouds
6,6,Puerto Ayora,EC,-0.74,-90.35,77.00,73,40,20.80,scattered clouds
7,7,Cape Town,ZA,-33.93,18.42,60.01,67,75,8.05,broken clouds
8,8,Lovozero,RU,68.00,35.01,30.79,94,94,21.39,overcast clouds
9,9,Vung Tau,VN,10.35,107.08,78.06,82,100,4.90,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Coahuayana,MX,18.73,-103.68,89.10,50,18,5.59,few clouds
2,2,Dulce Nombre De Culmi,HN,15.10,-85.53,86.00,74,75,1.12,broken clouds
3,3,Puerto Escondido,MX,15.85,-97.07,87.80,66,40,5.82,scattered clouds
4,4,Saint-Philippe,RE,-21.36,55.77,73.40,78,0,8.05,clear sky
6,6,Puerto Ayora,EC,-0.74,-90.35,77.00,73,40,20.80,scattered clouds
9,9,Vung Tau,VN,10.35,107.08,78.06,82,100,4.90,overcast clouds
12,12,Cabo San Lucas,MX,22.89,-109.91,86.00,61,5,6.93,clear sky
15,15,Vaini,TO,-21.20,-175.20,73.40,88,20,4.70,few clouds
16,16,Otavi,NaN,-19.65,17.33,86.65,16,95,7.65,overcast clouds
18,18,Hobyo,SO,5.35,48.53,81.18,80,23,14.43,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Coahuayana,MX,89.10,few clouds,18.73,-103.68,
2,Dulce Nombre De Culmi,HN,86.00,broken clouds,15.10,-85.53,
3,Puerto Escondido,MX,87.80,scattered clouds,15.85,-97.07,
4,Saint-Philippe,RE,73.40,clear sky,-21.36,55.77,
6,Puerto Ayora,EC,77.00,scattered clouds,-0.74,-90.35,
9,Vung Tau,VN,78.06,overcast clouds,10.35,107.08,
12,Cabo San Lucas,MX,86.00,clear sky,22.89,-109.91,
15,Vaini,TO,73.40,few clouds,-21.20,-175.20,
18,Hobyo,SO,81.18,few clouds,5.35,48.53,
19,Alyangula,AU,75.20,overcast clouds,-13.85,136.42,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
#test for number of no hotel names
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [17]:
# 7. Drop the rows where there is no Hotel Name
clean_hotel_df = hotel_df

#print
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Coahuayana,MX,89.10,few clouds,18.73,-103.68,Hotel Los Arcos
2,Dulce Nombre De Culmi,HN,86.00,broken clouds,15.10,-85.53,Hotel maria jose
3,Puerto Escondido,MX,87.80,scattered clouds,15.85,-97.07,Aldea del Bazar Hotel and Spa
4,Saint-Philippe,RE,73.40,clear sky,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
6,Puerto Ayora,EC,77.00,scattered clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
9,Vung Tau,VN,78.06,overcast clouds,10.35,107.08,Seaside Resort
12,Cabo San Lucas,MX,86.00,clear sky,22.89,-109.91,Hotel Tesoro Los Cabos
15,Vaini,TO,73.40,few clouds,-21.20,-175.20,Keleti Beach Resort
18,Hobyo,SO,81.18,few clouds,5.35,48.53,Osmani Hotel
19,Alyangula,AU,75.20,overcast clouds,-13.85,136.42,Groote Eylandt Lodge


In [19]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))